Retrieve info.json files

In [2]:
import os
import glob

sacred_dir = "sacred_cluster"
parent_dir = os.path.dirname(os.getcwd())
info_json_files = glob.glob(os.path.join(parent_dir,sacred_dir, "**/*info.json"), recursive=True)
print(len(info_json_files))

215


Check if file can be opened, is not empty and experiment  reached at least 2millions time steps.

In [4]:
import json
for i in info_json_files:
    try:
        with open(i, "r") as file:
            infos = json.load(file)
            if(len(infos.keys())==0):
                print("empty  dict", i)
            if infos["test_return_mean_T"][-1] < 2000000:
                # print("not long enough")
                # print(i)
                # print(infos["test_return_mean_T"][-1])
                print("rm -r "+str(os.path.basename(os.path.dirname(i))))

    except Exception as e:
        print("Error")
        print(e)
        print(i)
        print("rm -r "+str(os.path.basename(os.path.dirname(i))))
        # print("rm -r models/"+str(os.path.basename(os.path.dirname(i))))
        # print("rm -r tb_logs/"+str(os.path.basename(os.path.dirname(i))))
        # print("rm -r sacred/"+str(os.path.basename(os.path.dirname(i))))


rm -r ddmac_uc_50_struct_uc_50_3__2022-11-01_12-20-17
rm -r qmix_uc_10_struct_c_10_1__2022-11-02_14-26-56
rm -r ddmac_uc_50_struct_uc_50_10__2022-11-01_12-20-17
rm -r ddmac_uc_50_struct_uc_50_5__2022-11-01_12-20-17
rm -r ddmac_uc_50_struct_uc_50_1__2022-11-01_12-20-17


Count the number of each experiment

In [3]:
dict_exp={}
for i in  info_json_files:
    exp_name = os.path.basename(os.path.dirname(i))
    exp_name=exp_name.split("_")
    exp_name=[exp_name[0], exp_name[4], exp_name[5]]
    exp_name = "_".join(exp_name)
    dict_exp[exp_name] = dict_exp.get(exp_name, 0) + 1

for k, v in dict_exp.items():
    print(k, v)

qmix_uc_9 10
qmix_uc_3 10
ddmac_uc_10 10
ddmac_uc_50 6
qmix_uc_7 10
qmix_uc_10 11
ddmac_uc_5 10
qvmix_uc_7 10
qvmix_uc_5 10
ddmac_uc_7 10
ddmac_c_10 19
qmix_c_10 8
qvmix_uc_10 10
qvmix_uc_50 10
qvmix_uc_9 10
qmix_uc_5 10
qvmix_uc_3 10
qvmix_c_10 10
qmix_uc_50 11
ddmac_uc_3 10
ddmac_uc_9 10
